## Code to run the QAOA algorithm

In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from skopt import gp_minimize
import numpy as np
import math
import os
import logging
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import LinearEqualityToPenalty

import qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz

logger = logging.getLogger(__name__)

provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
provider.active_account()

{'name': 'challenge4_team1@qai-ventures.com',
 'token': 'rings-256.5a0wzAL00oeQOovqubPClxT9FJDezYum',
 'max_qubits': '256'}

## Define functions

In [2]:
# define the operator U(B, beta)
def Operator_UB(graph, beta,qc, q, n_qubits):
    for i in range(n_qubits): qc.rx(2*beta, q[i])


def generate_QUBO():
    """Code to generate the Pauli string from the QUBO formulation."""

    L = 1000 # power load to respect
    nb_units = 4
    p_max = 300
    p_min = 100

    qp = QuadraticProgram("optimisation")
    
    # Add variables
    for i in range(nb_units):
        qp.binary_var()

    # Inititalize the power to half-capacity
    power_units_init = [p_max/2] * nb_units
    qp.linear_constraint(linear=power_units_init, sense="==", rhs=L, name="lin_eq_load")
    print(qp.prettyprint())

    lineq2penalty = LinearEqualityToPenalty()
    qubo = lineq2penalty.convert(qp)
    print(qubo.prettyprint())
    
    pauli_operators, offset = qubo.to_ising()

    return pauli_operators, offset
     


# define the operator U(C,gamma)
def Operator_UC(pauli_op, gamma, qc, q, n_qubits):

    # implement the Pauli string representing the QUBO
    for paulis in pauli_op:
        # iterate through each pauli string
        for idx, gate in enumerate(paulis):
            if gate == "X":
                qc.x(q[idx])
            if gate == "Y":
                qc.y(q[idx])
            if gate == "Z":
                qc.z(q[idx])
        
        qc.cx(q[edge[0]], q[edge[1]])
        qc.rz(gamma*edge[2], q[edge[1]])
        qc.cx(q[edge[0]], q[edge[1]])




# Builds the QAOA state.
def qaoaState(x, graph, p, n_qubits, expectationValue=True, shots=1024):
    gammas = []
    betas = []
    # setup the gamma and beta array
    for i in range(len(x)//2):
        gammas.append(x[i*2])
        betas.append(x[(i*2)+1])
        
    # Create the quantum circuit
    q = QuantumRegister(n_qubits)
    c = ClassicalRegister(n_qubits)
    qc = QuantumCircuit(q, c)

    # First set the qubits in an equal superposition state
    for i in range(n_qubits):
        qc.h(q[i])

    # Apply the gamma and beta operators in repetition
    for i in range(p):
        # Apply U(C,gamma)
        Operator_UC(graph, gammas[i], qc, q, n_qubits)

        # Apply U(B, beta)
        Operator_UB(graph, betas[i],qc, q, n_qubits)

    # Measure the qubits
    for i in range(n_qubits):
        qc.measure(q[i], c[i])

    # Execute the circuit now
    job = backend.run(qc, shots)
    job.wait_for_final_state(0, 5, jobCallback)
    counts = job.result().get_counts()

    # decide what to return
    if (True == expectationValue):
        # Calculate the expectation value of the measurement.
        expectation = 0
        for bitstring in counts:
            probability = float(counts[bitstring]) / shots
            expectation += WeightOfCuts(bitstring,graph) * probability
        return( expectation )
    else:
        # Just return the counts return(counts)
        return(counts)

In [8]:
# from qiskit_aer.primitives import EstimatorV2 as Estimator
# from qiskit_aer.primitives import SamplerV2 as Sampler

pauli_op, offset = generate_QUBO()

capital = 1000 # total capital ($) to invest
nb_stocks = 5 # total number of stocks to consider

init_money_investment = [capital//nb_stocks]*nb_stocks
init_params_annealing = [np.pi, np.pi/2, np.pi, np.pi/2]
init_params = init_params_annealing + init_money_investment
print(f"Init parameters: {init_params}")

# generate the QAOA circuit
circuit = QAOAAnsatz(cost_operator=pauli_op, reps=2)
circuit.decompose().draw('mpl')


# exact_estimator = Estimator()

def cost_func(params, ansatz, hamiltonian, estimator):
    """ Return estimate of energy from estimator.

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, hamiltonian, params)
    cost = estimator.run([pub]).result()[0].data.evs
    
    return cost


print("Minimisation routine.")
# The simulation cannot be run on a circuit with qubits measurements
result = minimize(cost_func,
                 x0=init_params,
                 args=(circuit, cost_hamiltonian, exact_estimator),
                 method="COBYLA",
                 options={'maxiter':1000, 'disp':True})
print(result)

compute_eigenvalue = cost_func_vqe(params=result.x,
                                           ansatz=circuit,
                                           hamiltonian=pauli_op,
                                           estimator=exact_estimator)
print(f"Result eigenvalue: {compute_eigenvalue}")

Problem name: optimisation

Minimize
  0

Subject to
  Linear constraints (1)
    150*x0 + 150*x1 + 150*x2 + 150*x3 == 1000  'lin_eq_load'

  Binary variables (4)
    x0 x1 x2 x3

Problem name: optimisation

Minimize
  22500*x0^2 + 45000*x0*x1 + 45000*x0*x2 + 45000*x0*x3 + 22500*x1^2
  + 45000*x1*x2 + 45000*x1*x3 + 22500*x2^2 + 45000*x2*x3 + 22500*x3^2
  - 300000*x0 - 300000*x1 - 300000*x2 - 300000*x3 + 1000000

Subject to
  No constraints

  Binary variables (4)
    x0 x1 x2 x3

Init parameters: [3.141592653589793, 1.5707963267948966, 3.141592653589793, 1.5707963267948966, 200, 200, 200, 200, 200]
Minimisation routine.


NameError: name 'minimize' is not defined

## Classical Optimisation Loop
#### Optimisation of the $\gamma$, $\beta$ and $M$ parameters

In [ ]:
def run(input_data,solver_params,extra_args):

    logger.info("Starting solver execution...")
    
    p = 2                # Number of circuit layers
    n_calls = 100        # Optimization cycles
    n_random_starts = 2  # See scikit documentation
    dimensions = [] # Search space, place holder
    
    # Some example 2-regular graphs to play with, having equal weights.
    # Each graph is a list of edges and their weights.
    
    graph = [(0, 1, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
    #graph = [(0,1,1.0), (0,5,1.0), (1,2,1.0), (2,3,1.0), (3,4,1.0), (4,5,1.0)]
    
    # Find the number of qubits required.
    n_qubits = 0
    for i in range (len(graph)):
        for j in range (2):
            n_qubits = max(graph[i][j], n_qubits)
    n_qubits += 1
    
    # Construct the search space depending upon the circuit layers
    for i in range(p):
        dimensions.append((0,2*np.pi))
        dimensions.append((0,np.pi))
    d = tuple(dimensions)
    
    # setup the optimization function, as its negative
    f = lambda x: 1*qaoaState(x, graph, p, n_qubits)
    
    # Use the Bayesian optimization using Gaussian Processes from Scikit optimizer
    # to maximize the cost function (by minimizing its negative)
    res = gp_minimize(func=f,
            dimensions = d,
            n_calls=n_calls,
            n_random_starts=n_random_starts)
    
    # Fetch the optimal gamma and beta values
    x = res.x
    
    # Execute the qaoa state with the optimal gamma and beta values
    counts = qaoaState(x, graph, p, n_qubits, False)

    logger.info("End of solver execution.")
    
    return counts